In [1]:
import numpy as np
import os

from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.config import ExportFormat
from tflite_model_maker import model_spec
from tflite_model_maker import object_detector
from tensorflow_hub.module_v2 import load

from tflite_support import metadata

import tensorflow as tf
assert tf.__version__.startswith('2')
print(tf.__version__)

tf.get_logger().setLevel('ERROR')
from absl import logging
logging.set_verbosity(logging.ERROR)

c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(
c:\Users\Joshua\AppData\Local\Programs\Python\Python39\lib\site-packages\tensorflow_addons\utils\ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.10.0 and strictly below 2.13.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.9.0 and is not supported. 
Some things might work, some things might not.
If you were to encount

2.9.0


In [2]:
train_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/train',
    './project-6-at-2023-04-30-00-12-74270469/train',
    ['Counted_Objects', 'Not_Counted_Objects']
)

val_ds = object_detector.DataLoader.from_pascal_voc(
    './project-6-at-2023-04-30-00-12-74270469/validate',
    './project-6-at-2023-04-30-00-12-74270469/validate',
    ['Counted_Objects', 'Not_Counted_Objects']
)

In [3]:
print(train_ds.size)
print(val_ds.size)

108
27


In [4]:
#spec = model_spec.get('efficientdet_lite4')

spec = object_detector.EfficientDetSpec(
  model_name='efficientdet-lite4', 
  uri='https://tfhub.dev/tensorflow/efficientdet/lite4/feature-vector/2', 
  hparams={'max_instances_per_image': 8000})

In [5]:
model = object_detector.create(train_ds, model_spec=spec, batch_size=8, train_whole_model=True, epochs=200, validation_data=val_ds)

Epoch 1/200
13/13 [==============================] - 297s 19s/step - det_loss: 1.7707 - cls_loss: 1.1218 - box_loss: 0.0130 - reg_l2_loss: 0.1078 - loss: 1.8785 - learning_rate: 0.0090 - gradient_norm: 0.5569 - val_det_loss: 1.6668 - val_cls_loss: 1.0942 - val_box_loss: 0.0115 - val_reg_l2_loss: 0.1078 - val_loss: 1.7746
Epoch 2/200
13/13 [==============================] - 237s 18s/step - det_loss: 1.6243 - cls_loss: 1.0575 - box_loss: 0.0113 - reg_l2_loss: 0.1078 - loss: 1.7321 - learning_rate: 0.0100 - gradient_norm: 0.7343 - val_det_loss: 1.4718 - val_cls_loss: 0.9621 - val_box_loss: 0.0102 - val_reg_l2_loss: 0.1078 - val_loss: 1.5796
Epoch 3/200
13/13 [==============================] - 233s 18s/step - det_loss: 1.3636 - cls_loss: 0.8604 - box_loss: 0.0101 - reg_l2_loss: 0.1078 - loss: 1.4714 - learning_rate: 0.0100 - gradient_norm: 1.6414 - val_det_loss: 1.4613 - val_cls_loss: 1.0025 - val_box_loss: 0.0092 - val_reg_l2_loss: 0.1078 - val_loss: 1.5691
Epoch 4/200
13/13 [============

In [6]:
model.evaluate(train_ds)

2/2 [==============================] - 62s 24s/step



{'AP': 0.6509365,
 'AP50': 0.83439636,
 'AP75': 0.7604057,
 'APs': 0.0,
 'APm': 0.10709571,
 'APl': 0.6660897,
 'ARmax1': 0.18264596,
 'ARmax10': 0.49758843,
 'ARmax100': 0.7087705,
 'ARs': 0.0,
 'ARm': 0.105789475,
 'ARl': 0.724668,
 'AP_/Counted_Objects': 0.61508137,
 'AP_/Not_Counted_Objects': 0.6867916}

In [7]:
model.evaluate(val_ds)

1/1 [==============================] - 18s 18s/step



{'AP': 0.43869978,
 'AP50': 0.6440299,
 'AP75': 0.52064955,
 'APs': 0.0,
 'APm': 0.08415841,
 'APl': 0.4418128,
 'ARmax1': 0.13761169,
 'ARmax10': 0.3648812,
 'ARmax100': 0.53864443,
 'ARs': 0.0,
 'ARm': 0.08181818,
 'ARl': 0.54238963,
 'AP_/Counted_Objects': 0.38492018,
 'AP_/Not_Counted_Objects': 0.49247935}

In [8]:
model.export(export_dir='../obj_model_training/',tflite_filename='efd_4_uc_ha_bs8.tflite')

In [10]:
model.evaluate_tflite('./efd_4_uc_ha_bs8.tflite', train_ds)

108/108 [==============================] - 13205s 122s/step



{'AP': 0.47686073,
 'AP50': 0.60043156,
 'AP75': 0.5669065,
 'APs': 0.0,
 'APm': 0.0,
 'APl': 0.48802537,
 'ARmax1': 0.173631,
 'ARmax10': 0.41911706,
 'ARmax100': 0.50456715,
 'ARs': 0.0,
 'ARm': 0.0,
 'ARl': 0.51553816,
 'AP_/Counted_Objects': 0.39155817,
 'AP_/Not_Counted_Objects': 0.5621633}

In [9]:
model.evaluate_tflite('./efd_4_uc_ha_bs8.tflite', val_ds)

27/27 [==============================] - 3263s 121s/step



{'AP': 0.34401622,
 'AP50': 0.47713208,
 'AP75': 0.41936374,
 'APs': 0.0,
 'APm': 0.0,
 'APl': 0.34626484,
 'ARmax1': 0.13484038,
 'ARmax10': 0.32056138,
 'ARmax100': 0.3848064,
 'ARs': 0.0,
 'ARm': 0.0,
 'ARl': 0.3874473,
 'AP_/Counted_Objects': 0.25931346,
 'AP_/Not_Counted_Objects': 0.42871898}